### Notebook to scrub ebola dataset for Project 2

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
ebola_data_to_load = "Data/ebola_2014_2016_clean.csv"

# Read Population Data from "Estimates" sheet
ebola_data = pd.read_csv(ebola_data_to_load, encoding="ISO-8859-1")

In [3]:
ebola_data.head()

,Country,Date,No. of suspected cases,No. of probable cases,No. of confirmed cases,"No. of confirmed, probable and suspected cases",No. of suspected deaths,No. of probable deaths,No. of confirmed deaths,"No. of confirmed, probable and suspected deaths"
0,Guinea,2014-08-29,25.0,141.0,482.0,648.0,2.0,141.0,287.0,430.0
1,Nigeria,2014-08-29,3.0,1.0,15.0,19.0,0.0,1.0,6.0,7.0
2,Sierra Leone,2014-08-29,54.0,37.0,935.0,1026.0,8.0,34.0,380.0,422.0
3,Liberia,2014-08-29,382.0,674.0,322.0,1378.0,168.0,301.0,225.0,694.0
4,Sierra Leone,2014-09-05,78.0,37.0,1146.0,1261.0,11.0,37.0,443.0,491.0


In [4]:
# remove spaces and '.' in column names
ebola_data.columns = [c.replace(' ', '_') for c in ebola_data.columns]
ebola_data.columns = [c.replace('.', '') for c in ebola_data.columns]

# Find earliest and latest dates
print(ebola_data.Date.min())
print(ebola_data.Date.max())
ebola_data

2014-08-29
2016-03-23


,Country,Date,No_of_suspected_cases,No_of_probable_cases,No_of_confirmed_cases,"No_of_confirmed,_probable_and_suspected_cases",No_of_suspected_deaths,No_of_probable_deaths,No_of_confirmed_deaths,"No_of_confirmed,_probable_and_suspected_deaths"
0,Guinea,2014-08-29,25.0,141.0,482.0,648.0,2.0,141.0,287.0,430.0
1,Nigeria,2014-08-29,3.0,1.0,15.0,19.0,0.0,1.0,6.0,7.0
2,Sierra Leone,2014-08-29,54.0,37.0,935.0,1026.0,8.0,34.0,380.0,422.0
3,Liberia,2014-08-29,382.0,674.0,322.0,1378.0,168.0,301.0,225.0,694.0
4,Sierra Leone,2014-09-05,78.0,37.0,1146.0,1261.0,11.0,37.0,443.0,491.0
...,...,...,...,...,...,...,...,...,...,...
2480,Liberia,2016-03-23,5636.0,1879.0,3151.0,10666.0,NaN,NaN,NaN,4806.0
2481,Italy,2016-03-23,0.0,0.0,1.0,1.0,NaN,NaN,NaN,0.0
2482,Liberia,2016-03-23,0.0,3.0,2.0,5.0,NaN,3.0,1.0,4.0
2483,Nigeria,2016-03-23,0.0,1.0,19.0,20.0,0.0,1.0,7.0,8.0


In [6]:
# Add year column to dataframe
ebola_data['year'] = pd.DatetimeIndex(ebola_data['Date']).year
ebola_data

,Country,Date,No_of_suspected_cases,No_of_probable_cases,No_of_confirmed_cases,"No_of_confirmed,_probable_and_suspected_cases",No_of_suspected_deaths,No_of_probable_deaths,No_of_confirmed_deaths,"No_of_confirmed,_probable_and_suspected_deaths",year
0,Guinea,2014-08-29,25.0,141.0,482.0,648.0,2.0,141.0,287.0,430.0,2014
1,Nigeria,2014-08-29,3.0,1.0,15.0,19.0,0.0,1.0,6.0,7.0,2014
2,Sierra Leone,2014-08-29,54.0,37.0,935.0,1026.0,8.0,34.0,380.0,422.0,2014
3,Liberia,2014-08-29,382.0,674.0,322.0,1378.0,168.0,301.0,225.0,694.0,2014
4,Sierra Leone,2014-09-05,78.0,37.0,1146.0,1261.0,11.0,37.0,443.0,491.0,2014
...,...,...,...,...,...,...,...,...,...,...,...
2480,Liberia,2016-03-23,5636.0,1879.0,3151.0,10666.0,NaN,NaN,NaN,4806.0,2016
2481,Italy,2016-03-23,0.0,0.0,1.0,1.0,NaN,NaN,NaN,0.0,2016
2482,Liberia,2016-03-23,0.0,3.0,2.0,5.0,NaN,3.0,1.0,4.0,2016
2483,Nigeria,2016-03-23,0.0,1.0,19.0,20.0,0.0,1.0,7.0,8.0,2016


In [7]:
# How many occurrences of each country in dataset?
ebola_data['Country'].value_counts()

Liberia                     365
Sierra Leone                259
Guinea                      259
Nigeria                     255
Senegal                     254
United States of America    245
Mali                        243
Spain                       243
United Kingdom              221
Italy                       141
Name: Country, dtype: int64

In [9]:
# Group by country and year
ebola_1 = ebola_data.groupby(["Country", "year"], as_index = False).agg(
    {
        'No_of_confirmed_cases':'sum',    
        'No_of_confirmed_deaths': 'sum'
    }
)

ebola_1

,Country,year,No_of_confirmed_cases,No_of_confirmed_deaths
0,Guinea,2014,56295.0,31014.0
1,Guinea,2015,689976.0,428114.0
2,Guinea,2016,3351.0,2083.0
3,Italy,2015,140.0,0.0
4,Italy,2016,1.0,0.0
5,Liberia,2014,72254.0,11385.0
6,Liberia,2015,693322.0,15548.0
7,Liberia,2016,3162.0,4.0
8,Mali,2014,111.0,21.0
9,Mali,2015,1540.0,0.0


In [10]:
# Save scrubbed pop file to .csv
ebola_1.to_csv(r'Data/ebola_2014-2016.csv', index = False, header=True)